#### Imports

In [1]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Get data from .txt files

In [10]:
directory_path = '../data/new_jobs/'
files = os.listdir(directory_path)
count = 0
for i in files:
    count += 1
count

1343

In [20]:
items = []

remote_ratio = ['On-site', 'Remote', 'Hybrid']
employment_type = ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']
experience_level = ['Internship', 'Entry level', 'Associate', 'Mid-Senior level', 'Director', 'Executive']

for file_name in files:
    row = []
    job_id = file_name.replace('.txt', '')
    row.append(job_id)
    file_path = os.path.join(directory_path, file_name)
        
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            soup = BeautifulSoup(file.read(), 'html.parser')
            try: 
                container = soup.find_all('div', {'class': 'jobs-unified-top-card'})[0]
                image = container.find_all('img')[0].get('src')
                title = container.find_all('h1', {'class': 'job-details-jobs-unified-top-card__job-title'})[0].getText().strip()
                row.append(title)
                company_name = container.find_all('a', {'class': 'app-aware-link'})[1].getText().strip()
                row.append(company_name)
                state = container.find(string = lambda text: text and '· ' in text)
                row.append(state)
                line = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[0].getText().strip()
                line = line.split('\n')
                line = [i for i in line if i != '']
                line = [i for i in line if i != ' ']
                if len(line) == 4:
                    salary_range = line[0]
                    remote = line[1]
                    employment = line[2]
                    experience = line[3]
                    company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
                elif len(line) == 3:
                    if line[0] in remote_ratio and line[1] in employment_type and line[2] in experience_level:
                        salary_range = np.nan
                        remote = line[0]
                        employment = line[1]
                        experience = line[2]
                    if '$' in line[0] and line[1] in employment_type and line[2] in experience_level:
                        salary_range = line[0]
                        remote = np.nan
                        employment = line[1]
                        experience = line[2]
                    if '$' in line[0] and line[1] in remote_ratio and line[2] in employment_type:
                        salary_range = line[0]
                        remote = line[1]
                        employment = line[2]
                        experience = np.nan
                    company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
                elif len(line) == 2:
                    if '$' in line[0] and line[1] in employment_type:
                        salary_range = line[0]
                        remote = np.nan
                        employment = line[1]
                        experience = np.nan
                    if line[0] in remote_ratio and line[1] in employment_type:
                        salary_range = np.nan
                        remote = line[0]
                        employment = line[1]
                        experience = np.nan
                    if line[0] in employment_type and line[1] in experience_level:
                        salary_range = np.nan
                        remote = np.nan
                        employment = line[0]
                        experience = line[1]
                    company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
                row.append(salary_range)
                row.append(remote)
                row.append(employment)
                row.append(experience)
                row.append(company_size)
                row.append(image)
                items.append(row)
            except:
                print("next")

columns = ['job_id', 'job_title', 'company_name', 'company_state', 'salary_range', 'remote_ratio', 'employment_type', 'experience_level', 'company_size', 'image_link']

linkedin = pd.DataFrame(items, columns=columns)
linkedin.sample()

next
next
next
next
next
next


,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,image_link
145,3767242610,Senior Data Scientist,Warner Music Group,"· Los Angeles, CA ·","$140,000/yr - $200,000/yr",Remote,Full-time,NaN,"1,001-5,000 employees · Musicians",https://media.licdn.com/dms/image/C4E0BAQFEEde...


In [21]:
linkedin[linkedin['experience_level'] == 'Jornada completa']

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,image_link


In [22]:
linkedin.shape

(293, 10)

In [24]:
# linkedin.to_csv('../data/linkedin.csv', index=False)
linkedin.to_csv('../data/linkedin_boolean.csv', index=False)